In [1]:
import numpy as np
from math import pi
from scipy.linalg import solve
from scipy.special import jv

# basic periodic scatter information
from novice_stakes.periodic_scatter import Bragg

# complete reflection coefficent calculation modules to check results
from novice_stakes.periodic_scatter import CosineRs, QuadRs

In [14]:
# incident plane wave parameters
theta_inc = 15. * pi / 180.
c = 1500.
fc = 500.

# Sinusoidal paramters
H = 2.
L = 40.
K = 2 * pi / L

# discretize surface
decimation = 8  # integration lengths per acoustic wavelength
dx = c / (8 * fc)
numx = int(np.ceil(L / dx))
dx = L / numx
xper = np.arange(numx) * dx
z_wave = (H / 2) * np.cos(K * xper)
zp_wave = -(H * K / 2) * np.sin(K * xper)

# general considerations for periodic scatter calculations
num_eva = 10
bragg = Bragg(L)
qs = bragg.qvec(theta_inc, num_eva, fc)
a0, an, b0, bn = bragg.bragg_angles(theta_inc, qs, fc)

# source and receiver specifications
xsrc = 0
zsrc = -10
xrcr = 200
zrcr = -20

In [15]:
# Analytic solution of reflection coefficents specific to sinusoidal surface
a_inc = 1j ** qs * jv(qs, -b0 * H / 2)

qdiff = qs[None, :] - qs[:, None]
a_sca = 1j ** qdiff * jv(qdiff, bn[None, :] * H / 2)

# solve system of equation for reflection coefficents
rs_ana = solve(-a_sca, a_inc)
p_rfm_ana = bragg.p_sca(theta_inc, qs, fc, rs_ana, xsrc, zsrc, xrcr, zrcr)
p_rfm_ana

(-0.10821968411364119-0.3637673890258957j)

In [16]:
# KA provides a sanity check of solution
# it is most accurate for low surface slopes and high grazing angles
integrator = QuadRs(xper, z_wave, zp_wave)
rs_ka = integrator.ka(theta_inc, qs, fc)
p_ka = bragg.p_sca(theta_inc, qs, fc, rs_ka, xsrc, zsrc, xrcr, zrcr)
p_ka

(-0.04425435231266406-0.2839322912485266j)

In [17]:
# check naive notebook implimentation against module results
r_cos = CosineRs(H, L, c=c)
r1_ana = r_cos.rfm_1st(theta_inc, qs, fc)
np.max(np.abs(rs_ana - r1_ana))

0.0

In [18]:
bragg.r_energy(theta_inc, qs, fc, r1_ana)

1.0000000000000004

In [19]:
bragg.r_energy(theta_inc, qs, fc, rs_ka)

1.0029770464352001